> **Note:** In most sessions you will be solving exercises posed in a Jupyter notebook that looks like this one. Because you are cloning a Github repository that only we can push to, you should **NEVER EDIT** any of the files you pull from Github. Instead, what you should do, is either make a new notebook and write your solutions in there, or **make a copy of this notebook and save it somewhere else** on your computer, not inside the `sds` folder that you cloned, so you can write your answers in there. If you edit the notebook you pulled from Github, those edits (possible your solutions to the exercises) may be overwritten and lost the next time you pull from Github. This is important, so don't hesitate to ask if it is unclear.

## Use Connector class for accessing the internet
Even if logging is not important for the below exercises, get in the habit of using this class for connecting to the internet, to practice logging your activity. This will be expected in the final exam.

You should run `pip install scraping_class` to install the module to be used.

In [8]:
import scraping_class
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile)

# Exercise Set 6: Introduction to Web Scraping

In this Exercise Set we shall practice our webscraping skills utiilizing only basic python.  
We shall cover variations between static and dynamic pages and build. 

## Exercise Section 6.1: Scraping Jobnet.dk

This exercise you get to practice locating the request that the JavaScript sends to get the job data that it builds the joblistings from. You should use the **>Network Monitor<** tool in your browser.

Furthermore you practice spotting how the pagination is done, without clicking on the next page button, but instead changing a small parameter in the URL.

> **Ex. 6.1.1:** Go to  www.jobnet.dk and investigate the page. Figure our waht url you need to scrape to collect jobposting data. Sometimes this can be hard and requires you to inspect the page and look in the **>Network Monitor<** by filtering by XHR files.

> **Ex. 6.1.2.:** Use the `request` module to collect the first 20 results and unpack the relevant `json` data into a `pandas` DataFrame.

> **Ex. 6.1.3.:** Store the 'TotalResultCount' value for later use.

In [2]:
# [Answer to Ex. 6.1.1-3 here]

> **Ex. 6.1.4:** This exercise is about paging the results. We need to understand the websites pagination scheme. 

> Now scroll down the webpage and press the next page button. See how the parameters of the url changes as you turn the pages.

> **Ex. 6.1.5:** Design a`for` loop using the `range` function that changes this paging parameter in the URL. Use the TotalResultCount parameter from before to define the limits of the range function. Store these urls in a container. 

>**extra** Change the SortValue parameter from BestMatch to CreationDate, to make the sorting amendable to updating results daily.

*(HINT: See that the parameter is an offset and that this relates to the number of results pr. call made.)*

In [4]:
# [Answer to Ex. 6.1.4-5 here]

In [5]:
q = 'https://job.jobnet.dk/CV/FindWork/Search?SortValue=CreationDate&Offset=%d'

links = []
for offset in range(0,n_listings+20,20):
    url = q%offset
    links.append(url)

NameError: name 'n_listings' is not defined

> **Ex.6.1.6:** Pick 20 random links using the `random.sample()` function and collect them using the `Connector` class. Also use the `time.sleep()` function to limit the rate of your calls. Make sure to save the links already collected in a `set()` container to avoid having to reload links already collected. ***extra***: monitor the time left to completing the loop by using `tqdm.tqdm()` function.

> **Ex.6.1.7:** Load all the results into a DataFrame.

In [7]:
import random
import time
done = set()
data = []
import tqdm
for url in tqdm.tqdm(random.sample(links,20)):
    response,call_id = connector.get(url,'download_jobposting')
    if response.ok:
        d = response.json.loads()
    else:
        print('error')
    data += d['JobPositionPostings']
    time.sleep(0.5)
df = pd.DataFrame(data)
df.sample(5)

ValueError: Sample larger than population or is negative

In [9]:
d

NameError: name 'd' is not defined

## Exercise Section 6.2: Scraping Trustpilot.com
Now for a slightly more elaborate, yet still simple scraping problem. Here we want to scrape trustpilot for user reviews. This data is very nice since it provides free labeled data (rating) to train a machine learning model to understand positive and negative sentiment. 

Here you will practice crawling a website collecting the links to each company review page, and finally locate another behind the scenes JavaScript request that gets the review data in a neat json format.

> **Ex. 6.2.1:** Visit the https://www.trustpilot.com/ website and locate the categories page.
From this page you find links to company listings.

> **Ex. 6.2.2:**
Get the category page using the `requests` module and extract each link to a specific category page from the HTML. This can be done using the basic python `.split()` string method. Make sure only links within the ***/categories/*** section are kept, checking each string using the ```if 'pattern' in string``` condition. 

*(Hint: The links are relative. You need to add the domain name)*


In [8]:
# [Answer to Ex. 6.2.1-2]

In [11]:
url = 'https://www.trustpilot.com/categories'
response,call_id = connector.get(url,'mapping_categories')
if response.ok:
    html = response.text
else:
    print('error')
print(html)
links = set()
for link_loc in html.split('href="')[1:]:
    link = link_loc.split('"')[0]
    if '/categories/' in link:
        links.add(link)
print(len(links),list(links)[0]) # link is relative
links = ['https://www.trustpilot.com'+link for link in links]# add the domain to each link
links[:10]

<!DOCTYPE html><html lang="en-US"><head><meta charSet="UTF-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="shortcut icon" type="image/x-icon" href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/favicon.ico"/><link rel="manifest" href="/manifest.json"/><meta name="application-name" content="Trustpilot"/><meta name="theme-color" content="#000032"/><link rel="apple-touch-icon" sizes="180x180" href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/apple-touch-icon.png"/><link rel="icon" type="image/png" sizes="32x32" href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/favicon-32x32.png"/><link rel="icon" type="image/png" sizes="16x16" href="https://cdn.trustpilot.net/brand-assets/1.5.0/favicons/favicon-16x16.png"/><link rel="mask-icon" href="https://cdn.trustpilot.net/brand-assets/1.9.0/favicons/safari-pinned-tab.svg" color="#00b67a"/><meta name="apple-mobile-web-app-title" content="Trustpilot"/><meta name="msapplication-TileColor

['https://www.trustpilot.com/categories/professional_organizations',
 'https://www.trustpilot.com/categories/animals_pets',
 'https://www.trustpilot.com/categories/yoga_meditation',
 'https://www.trustpilot.com/categories/home_garden',
 'https://www.trustpilot.com/categories/insurance',
 'https://www.trustpilot.com/categories/ball_games',
 'https://www.trustpilot.com/categories/vehicle_repair_fuel',
 'https://www.trustpilot.com/categories/moving_storage',
 'https://www.trustpilot.com/categories/gaming',
 'https://www.trustpilot.com/categories/kids_family']

In [1]:
html.split('href="')[1:]

NameError: name 'html' is not defined

> **Ex. 6.2.3:** Get one of the category section links. Write a function to extract the links to the company review page from the HTML.

> **Ex. 6.2.4:** Figure out how the pagination is done, by following how the url changes when pressing the **next page**-button to obtain more company listings. Write a function that builds links to paging all the company listing results of each category. This includes parsing the number of subpages of each category and changing the correct parameter in the url.

(Hint: Find the maximum number of result pages, right before the next page button and make a loop change the page parameter of the url.)


In [12]:
#[Answer to Ex.6.2.3-4]

In [16]:
url = 'https://www.trustpilot.com/categories/art'
response, _ = connector.get(url,'mapping')
if response.ok:
    html = response.text
else:
    print('error')
def get_links(html):
    links = set() # define container
    for link_loc in html.split('href="')[1:]: # locate the start of a link
        link = link_loc.split('"')[0] # split at the end of the link
        links.add(link) # if it is: add it to the set container
    return links
def get_company_links(html):
    company_links = [link for link in get_links(html) if '/review/' in link] # check if the /review/ pattern is in the link
    return company_links
company_links = get_company_links(html)
print(len(company_links))
def get_all_category_pages(category_link):
    response, _ = connector.get(category_link,'mapping_categories')
    if response.ok:
        html = response.text
    else:
        print('error')
        return False
    links = get_links(html)
    # find the max_page.
    page_links = [link for link in links if '?page=' in link] # check if the paging parameter is in the link
    if len(page_links)==0: # no pages.
        return [category_link]
    n_pages = max([int(link.split('page=')[-1]) for link in page_links]) # extract the page value and take the max
    paging_links = [category_link] # define container and store the original result page
    q = category_link+'?page=%d' # define the varying parameter string.
    for num in range(2,n_pages+1): # build the links.
        paging_links.append(q%num)
    return paging_links
    max = paging_links
    
    print(max)

36


In [14]:
company_links

['/review/www.crystalcentral.com',
 '/review/stuff4crafts.com',
 '/review/www.icanvas.com',
 '/review/digitalzenify.com',
 '/review/sparklesmakeitspecial.com',
 '/review/www.stampanniething.com',
 '/review/www.canvasgicleeprinting.com',
 '/review/supremepawtrait.com',
 '/review/www.paleodirect.com',
 '/review/makeshirtsnow.com',
 '/review/modernpostcard.com',
 '/review/investivtg.com',
 '/review/paradiseawards.com',
 '/review/mixbook.com',
 '/review/h-fconsulting.com',
 '/review/outbyte.com',
 '/review/nametagcountry.com',
 '/review/tailorbrands.com',
 '/review/www.moo.com',
 '/review/www.handmadepiece.com',
 '/review/matboardandmore.com',
 '/review/thenightsky.io',
 '/review/candlemaking.com',
 '/review/nicebadge.com',
 '/review/www.refund123.com',
 '/review/www.fineartamerica.com',
 '/review/brandcrowd.com',
 '/review/jasper52.com',
 '/review/www.deviantart.com',
 '/review/www.greatbigcanvas.com',
 '/review/bestsuratsarees.com',
 '/review/koffee-express.com',
 '/review/zatista.com',


> **Ex. 6.2.5:** Loop through all categories and build the paging links using the above defined function.

> **Ex. 6.2.6:** Randomly pick one of category listing links you have generated, and get the links to the companies listed using the other function defined. 

> **Ex. 6.2.7:** Visit one of these links and inspect the **>Network Monitor<** to locate the request that loads the review data. Use the requests module to retrieve this link and unpack the json results to a pandas DataFrame.


In [15]:
#[Answer to Ex.6.2.5-7]

In [16]:
# ex. 6.2.5. Build the paging links

# It may take som time to run
# You'll find 1000+ links
company_listings = []
for link in links: 
    if 'support.trustpilot' in link:
        continue
    company_listings+=get_all_category_pages(link)
print('We need to visit %d company listing pages to collect all company addresses'%len(company_listings))

We need to visit 1256 company listing pages to collect all company addresses


JSONDecodeError: Expecting value: line 4 column 1 (char 3)

In [20]:
# ex. 6.2.6
company_links = get_company_links(random.choice(company_listings)) # use the above defined function

In [40]:
# ex. 6.2.7
direct_link = 'https://www.trustpilot.com/reviews/5f1c07401a5a69075066f63d/jsonld'
response, call_id = connector.get(direct_link, 'download_review')
d = response.json() # parse json using the build-in function.
df = pd.DataFrame(d)
df.head()

,@type,itemReviewed,author,datePublished,headline,reviewBody,reviewRating,publisher,inLanguage
@type,Review,Thing,Person,2020-07-25T10:19:44Z,Perfect in every way.,Perfect in every way.,Rating,Organization,en
name,Review,Byapptonly,Adrienne,2020-07-25T10:19:44Z,Perfect in every way.,Perfect in every way.,NaN,Trustpilot,en
url,Review,NaN,https://www.trustpilot.com/users/5f1c065af6abf...,2020-07-25T10:19:44Z,Perfect in every way.,Perfect in every way.,NaN,NaN,en
image,Review,NaN,https://user-images.trustpilot.com/5f1c065af6a...,2020-07-25T10:19:44Z,Perfect in every way.,Perfect in every way.,NaN,NaN,en
bestRating,Review,NaN,NaN,2020-07-25T10:19:44Z,Perfect in every way.,Perfect in every way.,5,NaN,en


Congratulations on coming this far. By now you are almost - still need to figure out how to page the reviews and to find the company ID in the html -, ready to deploy a scraper collecting all reviews on trustpilot. 
If you wanna see just how valuable such data could be visit the follow blogpost: https://blog.openai.com/unsupervised-sentiment-neuron/